# Business Problem/Objectives:
Metro Phoenix is a metropolitan area, centered on the city of Phoenix, that includes much of the central part of the state of Arizona. It is one of the fastest growing major metropolitan areas, gaining nearly 600,000 residents from 2010 to 2017. As of Census Bureau's 2017 population estimates, Metro ranks 11th largest Metropolitan Area in the nation with 4,737,270 residents. Metro Phoenix has historically been the center of the state's economy,ranks 5th in the nation in economic growth. The economic and population growth in the area draw attentions in the nation and from the world. 

An investment group is seeking opportunities to invest on short-term real estate rental business in Phoenix metro area in Arizona. One key factor to a successful short term rental business is the highly convenient surronding businesses. 
The investment group would like to have a general idea on phoenix metro area as how the business located in major suburb cities downtown area, are different suburb cities have same rental business potential based the business distributions, how is the business diversity and amenity and etc. Based on the request, we came across the following objectives:

1. the business diversities and amenities analysis in the metro phoenix area in order to help with their business decision.
2. the similarity of the suburb cites based on business distribution to avoid the repeated investments.

# Data:
The analysis will be performed on each individual suburb cities business data. In order to collect the business data, first of all, we extracted each city latitude and longitude from https://www.latlong.net/. Then based on the geographic location of each city, we use foursquare API to pull each city's business(ventures) data from https://foursquare.com/ The JSON file of each city's ventures data will be downloaded,transformed and appended into a single python dataframe, then the type, popularity of business and similarity of rental market based on each city's business type will be analyzed.


In [58]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('library imported')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

library imported


# Methodology

The methodology is the a guiding strategy for solving the problem, it provides a framework that what methods and processes that are used to obtain the answers and solve the problem. 
We have the following stages that composite the methodology of the project:
    1. Collect inspection data
    2. Explore and understand data
    3. Cleanse and prepare data
    4. Modeling

### 1. Collect data
We first select major surburb cities in metro phoenix area and collect the geographical latitude and longitude from https://www.latlong.net/

In [59]:
PHXData=[['Phoenix',33.4484,-112.0740],['Chandler',33.3062,-111.8413],['Scottsdale',33.4942,-111.9261 ],['Tempe',33.4255, -111.9400],['Paradise Valley',33.5312,-111.9426],
          ['Mesa',33.4152,-111.8315],['Gilbert',33.3528, -111.7890],['Fountain Hills',33.6042, -111.7257],['Glendale',33.5387, -112.1860],['Peoria',33.5806, -112.2374],
         ['Surprise',33.6292, -112.3679]]
#PHXData=[['Phoenix',33.4484,-112.0740],['Chandler',33.3062,-111.8413],['Scottsdale',33.4942,-111.9261 ],['Tempe',33.4255, -111.9400]]
PHXMetro=pd.DataFrame(PHXData, columns=['City', 'VLat','VLong'])
PHXMetro

,City,VLat,VLong
0,Phoenix,33.4484,-112.0740
1,Chandler,33.3062,-111.8413
2,Scottsdale,33.4942,-111.9261
3,Tempe,33.4255,-111.9400
4,Paradise Valley,33.5312,-111.9426
5,Mesa,33.4152,-111.8315
6,Gilbert,33.3528,-111.7890
7,Fountain Hills,33.6042,-111.7257
8,Glendale,33.5387,-112.1860
9,Peoria,33.5806,-112.2374


#### Make a function in order to parse the json file we pulled from foursquare API

In [60]:
#make a function to parse the json file
def get_venue_type(row):
    try:
        venue_type = row['categories']
    except:
        venue_type = row['venue.categories']
        
    if len(venue_type) == 0:
        return None
    else:
        return venue_type[0]['name']

#### In order to append all cities venues data into one data frame, a empty data frame is created, the foursquare API parameters are created. 

In [61]:
# @hidden_cell
CLIENT_ID = 'XBAB4MJFXP0DXHTRM3NMJC1DOA1O5RQTVV4EGZLJATMWG1GJ' # your Foursquare ID
CLIENT_SECRET = 'JYO2J0S2KVFRSUM325OQSXRMEVTTFPSWVRWYJLFL5VBE5FXD' # your Foursquare Secret

In [62]:
VERSION = '20180604'
LIMIT = 1000
radius=5000

In [63]:
columns=['city','name','categories','lat','lng']
ALLDF=pd.DataFrame(columns=columns)
ALLDF.head()

,city,name,categories,lat,lng


#### A loop will go through all suburb cities and call API then append each city's venue data into the data frame

In [64]:
# editing this to loop through all cities.
for index, row in PHXMetro.iterrows():
    print (row['City'])
    latitude=PHXMetro.get_value(index, 'VLat')
    longitude=PHXMetro.get_value(index,'VLong')
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
    results=requests.get(url).json()
#    results
    venues = results['response']['groups'][0]['items']
    PHXDF = json_normalize(venues) # flatten JSON
# filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    PHXDF =PHXDF.loc[:, filtered_columns]
# filter the category for each row
    PHXDF['venue.categories'] = PHXDF.apply(get_venue_type, axis=1)
# clean columns
    PHXDF.columns = [col.split(".")[-1] for col in PHXDF.columns]
# add city name
    PHXDF['city']=row['City']
    columnstitle=['city','name','categories','lat','lng']
    PHXDF=PHXDF.reindex(columns=columnstitle)
    ALLDF=ALLDF.append(PHXDF)

Phoenix


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Chandler
Scottsdale
Tempe
Paradise Valley
Mesa
Gilbert
Fountain Hills
Glendale
Peoria
Surprise


In [68]:
ALLDF.head()

,city,name,categories,lat,lng
0,Phoenix,Valley Bar,Music Venue,33.450120,-112.074103
1,Phoenix,Kimpton Hotel Palomar Phoenix Cityscape,Hotel,33.447423,-112.073284
2,Phoenix,Cornish Pasty Co,Pub,33.450226,-112.074109
3,Phoenix,Orpheum Theater,Theater,33.449253,-112.076644
4,Phoenix,The Kettle Black,Pub,33.448468,-112.072451


### 2. Explore and understand the busniess data
Understand what type of the business in different suburb city downtown area, what are the unique business types are available in each city

In [69]:
BUSINESSDIVERSITY = ALLDF[['city','categories']]
BUSINESSDIVERSITY['counter']=1
GROUP_BUSINESS=BUSINESSDIVERSITY.groupby(['city','categories'])['counter'].sum()
print(GROUP_BUSINESS)


city      categories                   
Chandler  American Restaurant              2
          Antique Shop                     1
          BBQ Joint                        1
          Bar                              2
          Baseball Field                   1
          Beer Bar                         1
          Breakfast Spot                   2
          Brewery                          2
          Burger Joint                     1
          Café                             3
          Candy Store                      1
          Chinese Restaurant               4
          Coffee Shop                      5
          Convenience Store                1
          Deli / Bodega                    1
          Department Store                 1
          Design Studio                    1
          Dessert Shop                     1
          Discount Store                   1
          Donut Shop                       1
          Fast Food Restaurant             2
          Gift 

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [67]:
!conda install -c conda-forge pandasql
from pandasql import sqldf

Solving environment: done

# All requested packages already installed.



#### Find count of unique business type of each suburb city.

In [70]:
UNIQUEBUSINESS = ALLDF[['city','categories']]
UNIQUEBUSINESS['ncount'] = UNIQUEBUSINESS.groupby('city')['categories'].transform('nunique')
UNIQUEBUSINESS.groupby('city')
#BUSINESSDIVERSITY
#print(BUSINESSDIVERSITY)
q="""SELECT DISTINCT city, ncount FROM UNIQUEBUSINESS;"""
pysqldf = lambda q: sqldf(q, globals())
a_df = pysqldf(q)
a_df

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,city,ncount
0,Phoenix,55
1,Chandler,55
2,Scottsdale,62
3,Tempe,57
4,Paradise Valley,53
5,Mesa,62
6,Gilbert,51
7,Fountain Hills,54
8,Glendale,54
9,Peoria,55


### 3. Prepare and Analyze data

In [72]:
ALLDF_onehot = pd.get_dummies(ALLDF[['categories']], prefix="", prefix_sep="")
ALLDF_onehot['city'] = ALLDF['city']
fixed_columns = [ALLDF_onehot.columns[-1]] + list(ALLDF_onehot.columns[:-1])
ALLDF_onehot=ALLDF_onehot[fixed_columns]
ALLDF_onehot.shape

(1085, 196)

#### We group rows by suburb cities and by taking the mean of the frequency of occurence of each category

In [73]:
city_group = ALLDF_onehot.groupby('city').mean().reset_index()
city_group

,city,ATM,Advertising Agency,Alternative Healer,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Chandler,0.00,0.00,0.00,0.020000,0.01,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.00,0.000000,0.020000,0.00,0.01,0.010000,0.01,0.00
1,Fountain Hills,0.00,0.00,0.00,0.054945,0.00,0.00,0.00,0.00,0.00,...,0.00,0.054945,0.00,0.000000,0.000000,0.00,0.00,0.010989,0.00,0.00
2,Gilbert,0.00,0.00,0.00,0.010000,0.00,0.00,0.00,0.01,0.00,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.01,0.00
3,Glendale,0.00,0.00,0.00,0.031915,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.00,0.010638,0.021277,0.00,0.00,0.000000,0.00,0.00
4,Mesa,0.00,0.00,0.00,0.020000,0.00,0.01,0.01,0.00,0.01,...,0.01,0.000000,0.00,0.000000,0.010000,0.01,0.00,0.000000,0.00,0.01
5,Paradise Valley,0.00,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.00,...,0.01,0.040000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00
6,Peoria,0.01,0.00,0.01,0.020000,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.00,0.010000,0.010000,0.00,0.00,0.000000,0.00,0.00
7,Phoenix,0.00,0.00,0.00,0.060000,0.00,0.01,0.00,0.05,0.01,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.00,0.00
8,Scottsdale,0.00,0.01,0.00,0.040000,0.00,0.00,0.00,0.00,0.01,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.00,0.01
9,Surprise,0.00,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.00,0.000000,0.010000,0.01,0.00,0.000000,0.01,0.00


#### Check the person correlation to exam the how different type of business relate to each other

In [74]:
city_group.corr()

,ATM,Advertising Agency,Alternative Healer,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
ATM,1.000000,-0.100000,1.000000,-0.217241,-0.100000,-0.149071,-0.100000,-0.120000,-0.239046,-0.100000,...,-0.149071,-0.146864,-0.100000,0.645086,0.140471,-0.149071,-0.100000,-0.345910,-0.193649,-0.193649
Advertising Agency,-0.100000,1.000000,-0.100000,0.159497,-0.100000,-0.149071,-0.100000,-0.120000,0.418330,-0.100000,...,-0.149071,-0.146864,-0.100000,-0.148984,-0.258560,-0.149071,-0.100000,0.277975,-0.193649,0.516398
Alternative Healer,1.000000,-0.100000,1.000000,-0.217241,-0.100000,-0.149071,-0.100000,-0.120000,-0.239046,-0.100000,...,-0.149071,-0.146864,-0.100000,0.645086,0.140471,-0.149071,-0.100000,-0.345910,-0.193649,-0.193649
American Restaurant,-0.217241,0.159497,-0.217241,1.000000,-0.217241,0.237765,-0.217241,0.455114,0.268700,-0.217241,...,0.097363,0.590059,-0.028872,-0.151294,-0.456802,-0.464246,-0.217241,0.299830,-0.663869,-0.055910
Antique Shop,-0.100000,-0.100000,-0.100000,-0.217241,1.000000,-0.149071,-0.100000,-0.120000,-0.239046,-0.100000,...,-0.149071,-0.146864,-0.100000,-0.148984,0.539502,-0.149071,1.000000,0.277975,0.516398,-0.193649
Arcade,-0.149071,-0.149071,-0.149071,0.237765,-0.149071,1.000000,0.670820,0.641006,0.623610,0.670820,...,0.388889,-0.218932,-0.149071,-0.222092,-0.088018,0.388889,-0.149071,-0.050636,-0.288675,0.240563
Arepa Restaurant,-0.100000,-0.100000,-0.100000,-0.217241,-0.100000,0.670820,1.000000,-0.120000,0.418330,1.000000,...,0.670820,-0.146864,-0.100000,-0.148984,0.140471,0.670820,-0.100000,-0.345910,-0.193649,0.516398
Art Gallery,-0.120000,-0.120000,-0.120000,0.455114,-0.120000,0.641006,-0.120000,1.000000,0.370521,-0.120000,...,-0.178885,-0.176237,-0.120000,-0.178781,-0.310272,-0.178885,-0.120000,0.333570,-0.090370,-0.232379
Art Museum,-0.239046,0.418330,-0.239046,0.268700,-0.239046,0.623610,0.418330,0.370521,1.000000,0.418330,...,0.133631,-0.351072,0.418330,-0.356140,-0.379609,0.133631,-0.239046,0.291644,-0.462910,0.810093
Arts & Crafts Store,-0.100000,-0.100000,-0.100000,-0.217241,-0.100000,0.670820,1.000000,-0.120000,0.418330,1.000000,...,0.670820,-0.146864,-0.100000,-0.148984,0.140471,0.670820,-0.100000,-0.345910,-0.193649,0.516398


#### Find top 10 most common type of business in each city

In [75]:
num_top_venues = 10

for city in city_group['city']:
    print("----"+city+"----")
    temp = city_group[city_group['city'] == city].T.reset_index()
    temp.columns = ['venue categories','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chandler----
     venue categories  freq
0  Mexican Restaurant  0.09
1         Pizza Place  0.08
2         Coffee Shop  0.05
3  Chinese Restaurant  0.04
4      Sandwich Place  0.03
5                Café  0.03
6    Sushi Restaurant  0.03
7          Taco Place  0.03
8  Italian Restaurant  0.03
9       Grocery Store  0.03


----Fountain Hills----
       venue categories  freq
0           Pizza Place  0.07
1   American Restaurant  0.05
2           Golf Course  0.05
3                 Trail  0.05
4    Mexican Restaurant  0.04
5         Grocery Store  0.03
6           Coffee Shop  0.03
7    Italian Restaurant  0.03
8  Fast Food Restaurant  0.03
9              Pharmacy  0.03


----Gilbert----
     venue categories  freq
0  Mexican Restaurant  0.08
1         Coffee Shop  0.07
2         Pizza Place  0.06
3       Grocery Store  0.04
4      Sandwich Place  0.04
5      Breakfast Spot  0.04
6    Sushi Restaurant  0.04
7        Burger Joint  0.03
8                 Gym  0.03
9  Italian Restaurant 

#### Define a function then generate a dataframe that contains 10 most common venues in each city

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['city'] = city_group['city']

for ind in np.arange(city_group.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_group.iloc[ind, :], num_top_venues)

city_venues_sorted

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chandler,Mexican Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Taco Place,Sandwich Place,Grocery Store,Café,Italian Restaurant,Sushi Restaurant
1,Fountain Hills,Pizza Place,Golf Course,American Restaurant,Trail,Mexican Restaurant,Italian Restaurant,Bank,Coffee Shop,Fast Food Restaurant,Pharmacy
2,Gilbert,Mexican Restaurant,Coffee Shop,Pizza Place,Grocery Store,Sushi Restaurant,Breakfast Spot,Sandwich Place,Italian Restaurant,Burger Joint,Gym
3,Glendale,Mexican Restaurant,Pizza Place,Convenience Store,Sandwich Place,Pharmacy,Fast Food Restaurant,American Restaurant,Donut Shop,Bar,Italian Restaurant
4,Mesa,Mexican Restaurant,Convenience Store,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Brewery,Greek Restaurant,Pool,Sports Bar
5,Paradise Valley,Hotel,American Restaurant,Pizza Place,Steakhouse,Trail,Golf Course,Resort,Restaurant,Grocery Store,Mexican Restaurant
6,Peoria,Pizza Place,Chinese Restaurant,Mexican Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Convenience Store,Bar,Fast Food Restaurant,Grocery Store
7,Phoenix,Coffee Shop,Hotel,Pizza Place,American Restaurant,Art Gallery,Bar,Music Venue,Cocktail Bar,Theater,History Museum
8,Scottsdale,Coffee Shop,Pizza Place,New American Restaurant,American Restaurant,Mexican Restaurant,Bar,Hotel,Café,Italian Restaurant,Sushi Restaurant
9,Surprise,Coffee Shop,Pharmacy,Pizza Place,Mexican Restaurant,Grocery Store,Italian Restaurant,Fast Food Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint


##### After analyzing the city data, we have better understanding how each city specilize in different type of business. Now we can proceed to modeling section. 

### 4. Modeling

We want to see based on the type of business, how similar short term rental investment environment will be among those cities to avoid repeated investment in similar cities. We choose K mean clustering as it is flexible to
adjust to the changes,compute fast and efficient.
We choose 4 clusters for suburb cites

In [91]:
# set number of clusters
kclusters = 4

city_group_clustering = city_group.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_group_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 3, 1, 3, 2, 1, 0, 0, 3], dtype=int32)

In [92]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = PHXMetro  

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_venues_sorted.set_index('city'), on='City')

city_merged # check the last columns!

,City,VLat,VLong,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Phoenix,33.4484,-112.0740,0,Coffee Shop,Hotel,Pizza Place,American Restaurant,Art Gallery,Bar,Music Venue,Cocktail Bar,Theater,History Museum
1,Chandler,33.3062,-111.8413,3,Mexican Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Taco Place,Sandwich Place,Grocery Store,Café,Italian Restaurant,Sushi Restaurant
2,Scottsdale,33.4942,-111.9261,0,Coffee Shop,Pizza Place,New American Restaurant,American Restaurant,Mexican Restaurant,Bar,Hotel,Café,Italian Restaurant,Sushi Restaurant
3,Tempe,33.4255,-111.9400,0,Coffee Shop,Pizza Place,Sandwich Place,Bar,Breakfast Spot,Brewery,Burger Joint,Chinese Restaurant,American Restaurant,Mexican Restaurant
4,Paradise Valley,33.5312,-111.9426,2,Hotel,American Restaurant,Pizza Place,Steakhouse,Trail,Golf Course,Resort,Restaurant,Grocery Store,Mexican Restaurant
5,Mesa,33.4152,-111.8315,3,Mexican Restaurant,Convenience Store,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Brewery,Greek Restaurant,Pool,Sports Bar
6,Gilbert,33.3528,-111.7890,3,Mexican Restaurant,Coffee Shop,Pizza Place,Grocery Store,Sushi Restaurant,Breakfast Spot,Sandwich Place,Italian Restaurant,Burger Joint,Gym
7,Fountain Hills,33.6042,-111.7257,2,Pizza Place,Golf Course,American Restaurant,Trail,Mexican Restaurant,Italian Restaurant,Bank,Coffee Shop,Fast Food Restaurant,Pharmacy
8,Glendale,33.5387,-112.1860,1,Mexican Restaurant,Pizza Place,Convenience Store,Sandwich Place,Pharmacy,Fast Food Restaurant,American Restaurant,Donut Shop,Bar,Italian Restaurant
9,Peoria,33.5806,-112.2374,1,Pizza Place,Chinese Restaurant,Mexican Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Convenience Store,Bar,Fast Food Restaurant,Grocery Store


# Result
We generate a map of the K mean clustering map for the client to better understand the result

In [93]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['VLat'], city_merged['VLong'], city_merged['City'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

# Discussion
The project is purely based on the business data downloaded from foursquare. The result may not reflect the short term rental business potentials objectively. In order to get more accurate result to help with the business decision, other factors like the average rents, population, average housing price and etc. of each city will need to be considered.

# Conclusion

From our analysis, we see: all cities have similar diversity of business and amount of the business. Among all cities, Mesa, Gilbert and Chandler geographically located close with each other and also with very similar type of businesses. City of Surprise though located far from these 3 cities, because of the nature of the busniess, 
it's still considered the similar investment environment. 
City of Paradise Valley and City of Fountain Hill are located apart, from the type of business, they both own trails and golf courses that are different than other cities, these 2 cities have similar investment potential.
Center cities City of Pheonix, City of Tempe and City of Scottsdale are similar type of investment environment; City of Peoria and City of Glendale are similar due to the similarity of businesses. 

Again, other factors like the average income, population, average rent, and etc. of each city should be combined with this analysis conclusion in order to achieve more accurate result, therefore to achieve wiser business decision. 
